# EC520 Image Processing and Communication
## Privacy Filter
Cameron Cipriano, Molly Housego

Depth-Driven Computational Imaging: Privacy Filter

The goal of the privacy filter is to identify people's faces in a scene and minimally distort them such that a facial recognition system will be unable to determine who it is. To perform minimal facial distortion, RGB and RGB+D Images were captured of a scene:
1. Facial Detection finds each face present in the image
2. Facial bounding boxes projected into RGB+D images to determine which points are facial pixels
3. Facial pixel coordinates reprojected to 2D image to define outline of face
4. Blur is applied to facial region and non-facial pixels are re-inserted to only distort the face.

Facial Detection implemented using a Haar Cascade Classifier

Facial recognition was implemented using a covariance matrix approach with Nearest Neighbor matching.

## Imports

In [1]:
import os
from typing import List, Tuple
from glob import glob
import csv
from itertools import product

import cv2 as cv
import numpy as np
import pptk
from scipy.linalg import eigh, svd
from tqdm.notebook import tqdm

## Helper Functions

In [2]:
def cleanDirectories(bbox_path, blur_path):
    bbox_image_names = glob(os.path.join(bbox_path, '*.jpg'))
    blur_image_names = glob(os.path.join(blur_path, '*.jpg'))

    for (bbox_img, blur_img) in zip(bbox_image_names, blur_image_names):
        try:
            os.remove(bbox_img)
            os.remove(blur_img)
        except OSError as e:
            print(f'Error: {e.strerror}')

def loadPointClouds(path):
    pointcloud_filenames = sorted(glob(os.path.join(path, '*.csv')))
    pointclouds = {}
    # Setup Progress bar
    iterations = tqdm(pointcloud_filenames, unit='Files')
    for file_idx, csvfile in enumerate(iterations):
        # each point cloud is 39,963 points, each with xyz and rgb values
        pointclouds[file_idx] = {'points': np.zeros((39963,3)), 'colors': np.zeros((39963,3))}
        with open(csvfile, newline='') as pointcloud_file:
            iterations.set_description(f"Parsing: '{pointcloud_file.name}'")
            point_reader = csv.reader(pointcloud_file)
            for point_idx, point_vals in enumerate(point_reader):
                # xyx for current point cloud
                pointclouds[file_idx]['points'][point_idx, 0] = point_vals[0]
                pointclouds[file_idx]['points'][point_idx, 1] = point_vals[1]
                pointclouds[file_idx]['points'][point_idx, 2] = point_vals[2]

                # RGB for current point cloud
                pointclouds[file_idx]['colors'][point_idx, 0] = point_vals[3]
                pointclouds[file_idx]['colors'][point_idx, 1] = point_vals[4]
                pointclouds[file_idx]['colors'][point_idx, 2] = point_vals[5]
    
    return pointclouds

def loadImages(path):
    # Read Images containing faces
    image_filenames = sorted(glob(os.path.join(path, '*.jpg')))
    
    # Setup Progress bar
    image_iterations = tqdm(image_filenames, unit='Images')
    
    images = []
    for filename in image_iterations:
        image_iterations.set_description(f"Loading: '{filename}'")
        raw_img = cv.imread(filename)

        img_scale = 512 / raw_img.shape[0]
        scaled_width  = int(img_scale * raw_img.shape[1])
        scaled_height = int(img_scale * raw_img.shape[0])

        images.append(cv.resize(raw_img, (scaled_width, scaled_height), interpolation=cv.INTER_AREA))
    
    return np.array(images)

## Facial Recognition - Implementation of Region Covariance Detector

In [85]:
def calc_distance(x: np.ndarray, y: np.ndarray) -> float:
    Vx, Dx, V_Tx = svd(x)
    Vy, Dy, V_Ty = svd(y)

    Dx_log = np.log10(Dx)
    Dy_log = np.log10(Dy)

    log_C_x = Vx @ np.diag(Dx_log) @ V_Tx
    log_C_y = Vy @ np.diag(Dy_log) @ V_Ty

    return np.linalg.norm((log_C_x - log_C_y))

class RegionCovarianceDetector:
    """
    Object Detector Using Region Covariance

    Parameters
    ----------
    coord : bool, optional (default=True)
        Whether use coordinates as features or not.

    color : bool, optional (default=True)
        Whether use color channels as features or not.

    intensity : bool, optional (default=False)
        Whether use intensity as feature or not.

    first_order : bool, optional (default=True)
        Scharr Filter applied to intensity image, first order derivative. If False, no filters are used.

    second_order : bool, optional (default=True)
        Scharr Filter applied to intensity image, second order derivative. If False, no filters are used.

    ratio : float, optional (default=1.15)
        Scaling factor between two consecutive scales of the search window size and step size.

    step : int, optional (default=3)
        The minimum step size.

    n_scales : int, optional (default=9)
        The number of scales of the windows.

    eps : float, optional (default=1e-16)
        Small number to keep covariance matrices in SPD.

    Attributes
    ----------
    object_shape : (int, int)
        The object's shape.

    object_covariance : np.ndarray, shape (n_features, n_features)
        Covariance matrix of the object.
    """

    def __init__(
            self,
            coord: bool = True,
            color: bool = True,
            intensity: bool = False,
            first_order: bool = True,
            second_order: bool = True,
            ratio: float = 1.15,
            step: int = 3,
            n_scales: int = 9,
            eps: float = 1e-16
    ):
        self.coord = coord
        self.color = color
        self.intensity = intensity
        self.first_order = first_order
        self.second_order = second_order
        self.ratio = ratio
        self.step = step
        self.n_scales = n_scales
        self.eps = eps

    def extract_features(self, img: np.ndarray) -> List[np.ndarray]:
        """
        Extract image features.

        Parameters
        ----------
        img : np.ndarray, shape (h, w, c)
            uint8 RGB image

        Returns
        -------
        features : a list of np.ndarray
            Features such as intensity, its gradient and so on.
        """
        h, w, c = img.shape[:3]
        intensity = cv.cvtColor(img, cv.COLOR_BGR2HSV)[:, :, 2] / 255.
        features = list()

        # use coordinates
        if self.coord:
            features.append(np.tile(np.arange(w, dtype=float), (h, 1)))
            features.append(np.tile(np.arange(h, dtype=float).reshape(-1, 1), (1, w)))

        # use color channels
        if self.color:
            for i in range(c):
                features.append(img[:, :, i].astype(float) / 255.)

        # use intensity
        if self.intensity:
            features.append(intensity)

        # use 1st-order derivatives of x and y of intensity image
        if self.first_order:
            first_order_x = np.abs(cv.Scharr(intensity, cv.CV_16S, 1, 0))
            first_order_y = np.abs(cv.Scharr(intensity, cv.CV_16S, 0, 1))
            features.extend([first_order_x, first_order_y])

        # use 2nd-order derivatives of x and y of intensity image
        if self.second_order and self.first_order:
            features.append(np.abs(cv.Scharr(first_order_x, cv.CV_16S, 1, 0)))
            features.append(np.abs(cv.Scharr(first_order_y, cv.CV_16S, 0, 1)))

        return features

    def calc_integral_images(self, img: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Calculate integral images.

        Parameters
        ----------
        img : np.ndarray, shape (h, w, c)
            uint8 RGB image

        Returns
        -------
        P : np.ndarray, shape (h+1, w+1, n_features)
            First order integral images of features.

        Q : np.ndarray, shape (h+1, w+1, n_features, n_features)
            Second order integral images of features.
        """
        h, w = img.shape[:2]
        features = self.extract_features(img)
        length = len(features)

        # first order integral images
        P = cv.integral(np.array(features).transpose((1, 2, 0)))

        # second order integral images
        Q = cv.integral(
            np.array(list(map(lambda x: x[0] * x[1], product(features, features)))).transpose((1, 2, 0))
        )
        Q = Q.reshape(h + 1, w + 1, length, length)
        return P, Q

    def calc_covariance(self, P: np.ndarray, Q: np.ndarray, pt1: Tuple[int, int], pt2: Tuple[int, int]) -> np.ndarray:
        """
        Calculate covariance matrix from integral images.

        Parameters
        ----------
        P : np.ndarray, shape (h+1, w+1, n_features)
            First order integral images of features.

        Q : np.ndarray, shape (h+1, w+1, n_features, n_features)
            Second order integral images of features.

        pt1 : (int, int)
            Left top coordinate.

        pt2 : (int, int)
            Right bottom coordinate.

        Returns
        -------
        covariance : np.ndarray, shape (n_features, n_features)
            Covariance matrix.
        """
        x1, y1 = pt1
        x2, y2 = pt2
        q = Q[y2, x2] + Q[y1, x1] - Q[y1, x2] - Q[y2, x1]
        p = P[y2, x2] + P[y1, x1] - P[y1, x2] - P[y2, x1]
        n = (y2 - y1) * (x2 - x1)
        covariance = np.abs((q - np.outer(p, p) / n) / (n - 1)) + (self.eps * np.identity(P.shape[2]))
        return covariance

    def set_search_object(self, img: np.ndarray) -> Tuple[np.ndarray, Tuple[int, int]]:
        """
        Calculate the object covariance matrix.

        Parameters
        ----------
        img : np.ndarray, shape (h, w, c)
            uint8 RGB image

        Returns
        -------
        : Fitted detector.
        """
        h, w = img.shape[:2]
        P, Q = self.calc_integral_images(img)

        # normalize about coordinates
        if self.coord:
            for i, size in enumerate((w, h)):
                P[:, :, i] /= size
                Q[:, :, i] /= size
                Q[:, :, :, i] /= size

        # calculate covariance matrix
        obj_cov = self.calc_covariance(P, Q, (0, 0), (w, h))
        obj_shape = (h, w)

        return obj_cov, obj_shape

    def find_object(self, object: Tuple[np.ndarray, Tuple[int, int]], target_img: np.ndarray) -> Tuple[Tuple[int, int], Tuple[int, int], float]:
        """
        Compute object's position in the target image.

        Parameters
        ----------
        object : (np.ndarray, (h, w))
            Object's representative covariance matrix and its height and width 

        target_img : np.ndarray, shape (h, w, c)
            uint8 RGB image

        Returns
        -------
        pt1 : (int, int)
            Left top coordinate.

        pt2 : (int, int)
            Right bottom coordinate.

        score : float
            Dissimilarity of object and target covariance matrices.
        """
        target_h, target_w = target_img.shape[:2]
        obj_h, obj_w = object[1]
        P, Q = self.calc_integral_images(target_img)

        # search window's shape and step size
        end = (self.n_scales + 1) // 2
        start = end - self.n_scales
        shapes = [(int(obj_h * self.ratio ** i), int(obj_w * self.ratio ** i)) for i in range(start, end)]
        steps = [int(self.step * self.ratio ** i) for i in range(self.n_scales)]

        level_iterations = tqdm(zip(shapes, steps), total=len(shapes))
        level_iterations
        distances = list()
        for level, (shape, step) in enumerate(level_iterations):
            level_iterations.set_description(f'Level: {level}, Shape: {shape}, Step: {step}')
            p_h, p_w = shape
            p_P, p_Q = P.copy(), Q.copy()

            # normalize about coordinates
            if self.coord:
                for i, size in enumerate((p_w, p_h)):
                    p_P[:, :, i] /= size
                    p_Q[:, :, i] /= size
                    p_Q[:, :, :, i] /= size

            distance = list()
            y1, y2 = 0, p_h
            while y2 <= target_h:
                dist = list()
                x1, x2 = 0, p_w
                while x2 <= target_w:
                    # calculate covariance matrix
                    p_cov = self.calc_covariance(p_P, p_Q, (x1, y1), (x2, y2))

                    # jump horizontally
                    x1 += step
                    x2 += step

                    # calculate dissimilarity of two covariance matrices
                    dist.append(calc_distance(object[0], p_cov))

                # jump vertically
                y1 += step
                y2 += step
                distance.append(dist)
            distances.append(np.array(distance))

        # choose the most similar window
        min_dist_indices = list(map(np.argmin, distances))
        min_dist_index = int(np.argmin([dist.flatten()[i] for i, dist in zip(min_dist_indices, distances)]))
        min_step = steps[min_dist_index]
        min_shape = shapes[min_dist_index]
        min_index = min_dist_indices[min_dist_index]
        b_h, b_w = distances[min_dist_index].shape

        pt1 = ((min_index % b_w) * min_step, (min_index // b_w) * min_step)
        pt2 = (pt1[0] + min_shape[1], pt1[1] + min_shape[0])
        score = distances[min_dist_index].flatten()[min_index]
        
        return pt1, pt2, score

## Main Algorithm Functions

In [82]:
def detectFaces(images, classifier):
    """
    Function for detecting faces

    Returns list of rectangles
    """
    detection_iterations = tqdm(images, unit='Image')

    face_bboxes = {}
    detected_face_visuals = []
    for img_idx, frame in enumerate(detection_iterations):
        detection_iterations.set_description('Detecting faces')
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        # Histogram equalization to improve contrast and make grayscale image more uniform
        frame_gray = cv.equalizeHist(frame_gray)

        #-- Detect faces
        # faces = classifier.detectMultiScale(frame_gray, scaleFactor=1.01, minNeighbors=7, minSize=(175, 175), maxSize=(300, 300), flags=cv.CASCADE_SCALE_IMAGE)
        faces = classifier.detectMultiScale(frame_gray, scaleFactor=1.05, minNeighbors=6, minSize=(30, 30), maxSize=(75, 75), flags=cv.CASCADE_SCALE_IMAGE)
        visual = frame.copy()
        for (x,y,w,h) in faces:
            # COLOR IS BGR!
            visual = cv.rectangle(visual, (x, y), (x + w, y + h), (0, 0, 255), 2)

        face_bboxes[img_idx] = faces
        detected_face_visuals.append(visual)

    return np.array(detected_face_visuals), face_bboxes

def recoverFaceOutlines(images, bboxes, P_mat, pointclouds):
    pointcloud_viewer = pptk.viewer(pointclouds[2]['points'], pointclouds[2]['colors']/255)
    pointcloud_viewer.set(lookat=np.zeros((3,1)), phi=-np.pi/2, theta=0, point_size=2)
    pointcloud_viewer.wait()
    pointcloud_viewer.close()

def blurFaceOutlines():
    # blurLevels = np.ones((1, images.shape[1]))
    # images_with_blurred_faces = np.empty(())
    # for img_idx, blurLevel in enumerate(blurLevels):
    #     images_with_blurred_faces[img_idx] = blurFaces(images_with_faces, face_indices, blurLevel)
    pass

def generateFacialRecognitionDatabase(path: str, face_detector, face_recognizer: RegionCovarianceDetector) -> Tuple[np.ndarray, np.ndarray]:
    image_filenames = sorted(glob(os.path.join(path, '*_frontal.jpg')))
    image_iterations = tqdm(image_filenames, unit='Images')
    
    images = []
    for filename in image_iterations:
        image_iterations.set_description(f'Loading templating images...')
        
        raw_img = cv.imread(filename)

        img_scale = 512 / raw_img.shape[0]
        scaled_width  = int(img_scale * raw_img.shape[1])
        scaled_height = int(img_scale * raw_img.shape[0])

        images.append(cv.resize(raw_img, (scaled_width, scaled_height), interpolation=cv.INTER_AREA))
    
    images = np.array(images)
    
    # Extract face templates to be used for facial recognition
    face_templates = []
    _, face_bboxes = detectFaces(images, face_detector)
    for (img_idx, boxes) in face_bboxes.items():
        for (x, y, w, h) in boxes:
            face_templates.append(images[img_idx][y:y+h, x:x+w, :])
    
    template_iterations = tqdm(face_templates, unit='Templates')
    template_iterations.set_description('Defining face templates')
    database = [face_recognizer.set_search_object(template) for template in template_iterations]

    print('Constructed facial recognition database!')
    return database

def blurRectangularRegion(frame, regions):
    for region in regions:
        (x, y, w, h) = region
        ROI = frame[y:y+h, x:x+w]
        blur = cv.GaussianBlur(ROI, (51, 51), 5)
        frame[y:y+h, x:x+w] = blur

    return frame

## Runner

In [87]:
input_img_path = 'images/jpg'
blurred_img_path = 'images/jpg/blurred'
bbox_img_path = 'images/jpg/bounding_boxes'
calibration_img_path = 'images/jpg/calibration'
pointcloud_path = 'pointclouds'


# Make sure output directories are empty
cleanDirectories(bbox_img_path, blurred_img_path)

# Load in the Facial Detection classifiers
face_cascade_alt2 = cv.CascadeClassifier('cascades/haarcascade_frontalface_alt2.xml')

# Create Facial Recognizer
face_recognizer = RegionCovarianceDetector()

database                       = generateFacialRecognitionDatabase(input_img_path, face_cascade_alt2, face_recognizer)
images                         = loadImages(input_img_path)
pointclouds                    = loadPointClouds(pointcloud_path)
K_camera                       = calibrate_camera(calibration_img_path)
images_with_faces, face_bboxes = detectFaces(images, face_cascade_alt2)
# output_tbd                     = recoverFaceOutlines(images, face_bboxes, None, pointclouds)
# blurred_images                 = blurFaceOutlines(images, face_bboxes, output_tbd)



  0%|          | 0/8 [00:00<?, ?Images/s]

  0%|          | 0/8 [00:00<?, ?Image/s]

  0%|          | 0/8 [00:00<?, ?Templates/s]

Constructed facial recognition database!


  0%|          | 0/9 [00:00<?, ?it/s]

-1